# This jupyter notebook...
This is a help to the quality assessment of the beads of an experiment

## Precision
Please note that for the detection of `missing` and `fixed` we have used several criteria based on the detected errors per bead. 

We observed from the analysis over 2 data sets of sequencing experiments that the criteria to detect *fixed beads* is precise at around 98%. This means that, for 100% of detected fixed beads, around 98% present the characteristic behavior of the missing beads, observable on the plot of the cycles of the bead. 
For the *missing beads*, the precision is about 80%. This means that, for 100% of detected missing beads, around 80% of them are missing on the field of view. 

## Bead Status
We will define the `status` of the bead mainly by the fact that the bead can be:
* `missing`
* `fixed`
* present one or more errors `errors>=1`
* not present errors at all `noErrors`

Note that a bead that does not present errors **is not** automatically a useful bead. 

# How to use it ?
Copy this jupyter notebook and then type the path of your files and then execute the cells to get information about the quality of the beads for your experiment.


In [ ]:
#import scripting
import sys
sys.path.insert(0, "../build")
from scripting import *

import beadquality as bq
#import pandas as pd

We can see what are the bounds set by default for the `cleaning` module of trackanalysis. These values will be used in the quality analysis.

In [ ]:
Tasks.cleaning().__dict__

## Path
Put the path of your file in the next cell

In [ ]:
#Load the tracks, put the path of the data folder
tracks = TracksDict("../../Documents/local_test_data/tests/ssHP6_Sequencing_20171220/*/*.trk", match=".*_043_(\w\w\w)_.*")


## Summary Table 
A summary table of number of **fixed**, **missing** and **not fixed/missing** beads for each of the tracks in your experiments.

A plot presenting the evolution of the number of **fixed**, **missing** and **not fixed/missing** beads.

In [ ]:
dfmsg = tracks.cleaning.messages().reset_index()

In [ ]:
resume = bq.resume_tracks_quality(tracks, dfmsg)
resume.table

In [ ]:
%%output backend='bokeh'
%%opts Points[tools=['hover'], show_grid=True] (marker='o' size=5)
%%opts Points.ok (color='blue')
%%opts Curve.ok (color='blue')
%%opts Points.fixed (color='orange')
%%opts Curve.fixed (color='orange')
%%opts Points.missing (color='red')
%%opts Curve.missing (color='red',line_dash='dashed')

bq.evolution_missing_fixed(resume.table,tracks)

## Summary of beads by Status
A heat map representing the percentage of beads by their status, per track. Use the attribute `table` to get a table that you can copy/paste in another document

In [ ]:
order_tracks_chrono = pd.Series(list(tracks.dataframe().set_index('key').sort_values(by='modification').index.values))


In [ ]:
df_resumeBeadsQuality = bq.resume_bead_quality(tracks, dfmsg, resume.status, order_tracks_chrono)


In [ ]:
tracks_status = bq.heatmap_tracks_status(df_resumeBeadsQuality.res,order_tracks_chrono,pc=False,order='chrono')
plt.show()

In [ ]:
tracks_status.table

## Bead Status per track
A detailed plot of bead per track showing its status

In [ ]:
plot = bq.heatmap_status(df_resumeBeadsQuality.res,order_tracks_chrono.values, bq.order_bead_best(df_resumeBeadsQuality))
from bokeh.io import show
show(plot)

## Flow by Status from initial to final track
A flow chart showing the evolution of the status of the beads from one track to another (by default from first to last track)

In [ ]:
df_typeError = bq.typeError(df_resumeBeadsQuality,tracks.availablebeads(),order_tracks_chrono)
flow_diagram = bq.flowBeads(df_typeError,first_track=None, last_track=None)
flow_diagram.show()